In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../../src/')

import os

from aind_vr_foraging_analysis.utils import parse, processing, plotting_utils as plotting, AddExtraColumns

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from pathlib import Path

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = r'Z:\scratch\vr-foraging\data'
data_path = r'../../../data/'

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'
color4='#e7298a'
odor_list_color = [color1, color2, color3]
color_dict = {0: color1, 1: color2, 2: color3}
color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3, 
                    '2-Heptanone' : color2, 'Methyl Acetate': color1, 'Fenchone': color3, '2,3-Butanedione': color4,
                    'Methyl Butyrate': color1}
# color_dict_label = {'Ethyl Butyrate': '#d95f02', 'Alpha-pinene': '#1b9e77', 'Amyl Acetate': '#7570b3', 
#                     '2-Heptanone' : '#1b9e77', 'Methyl Acetate': '#d95f02', 'Fenchone': '#7570b3', '2,3-Butanedione': '#e7298a'}
dict_odor = {}
rate = -0.12
offset = 0.6
dict_odor['Ethyl Butyrate'] = {'rate':rate, 'offset':offset, 'color': '#d95f02'}
dict_odor['Methyl Butyrate'] = {'rate':rate, 'offset':0.9, 'color': '#d95f02'}
dict_odor['Alpha-pinene'] = {'rate':rate, 'offset':offset, 'color': '#1b9e77'}
dict_odor['Amyl Acetate'] = {'rate':rate, 'offset':offset, 'color': '#7570b3'}
dict_odor['Methyl Acetate'] = {'rate':rate, 'offset':offset, 'color': color1}
dict_odor['2,3-Butanedione'] = {'rate':rate, 'offset':offset, 'color': color4}
dict_odor['Fenchone'] = {'rate':rate, 'offset':offset, 'color': '#7570b3'}
dict_odor['2-Heptanone'] = {'rate':rate, 'offset':offset, 'color': '#7570b3'}

# Define exponential function
def exponential_func(x, a, b):
    return a * np.exp(b * x)

def format_func(value, tick_number):
    return f"{value:.0f}"

results_path = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\experiments\batch 4 - manipulating cost of travelling and global statistics\results'


## Reversal experiments

In [ ]:
date = datetime.date.today()
date_string = "1/21/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

In [ ]:
summary_df = pd.DataFrame()
all_epochs = pd.DataFrame()
cum_trial_summary = pd.DataFrame()

for mouse in ['699894','694569', '690164']:
    session_n = 0
    previous_experiment = 0
    control_experiment = 0
    directory = os.path.join(base_path, mouse)
    files = os.listdir(os.path.join(base_path, mouse))

    sorted_files = sorted(files, key=lambda x: datetime.datetime.strptime(x[-15:-7], "%Y%m%d").date(), reverse=False)
    
    # All this segment is to find the correct session without having the specific path
    for file_name in sorted_files:

        print(mouse, file_name)
        # Find specific session sorted by date
        session = file_name[-15:-7]
        if datetime.datetime.strptime(session, "%Y%m%d").date() < date:
            continue
            
        # Recover data streams
        session_path = os.path.join(base_path, mouse, file_name)
        session_path = Path(session_path)
        try:
            data = parse.load_session_data(session_path)
        except:
            print('Error loading data', file_name)

        # Parse data
        try:
            if 'environmentStatistics' in data['config'].streams.TaskLogic.data:
                print(data['config'].streams.TaskLogic.data['environmentStatistics']['patches'][0]['patchRewardFunction'])
                print(data['config'].streams.TaskLogic.data['environmentStatistics']['patches'][1]['patchRewardFunction'])
                print(data['config'].streams.TaskLogic.data['environmentStatistics']['patches'][2]['patchRewardFunction'])
            else:
                print(data['config'].streams.TaskLogic.data['environment_statistics']['patches'][0]['reward_specification']['reward_function'])
                print(data['config'].streams.TaskLogic.data['environment_statistics']['patches'][1]['reward_specification']['reward_function'])
                print(data['config'].streams.TaskLogic.data['environment_statistics']['patches'][2]['reward_specification']['reward_function'])
        except:
            continue

In [ ]:
base_path = 'Z:/scratch/vr-foraging/data/'
batch = 2
summary_df = pd.DataFrame()
# mouse_list = ['694569','694569','694569', '694569','694569',
#               '690164', '690164', '690164','690164', '690164', 
#               '699894', '699894','699894', '699894', '699894']
# file_list = ['20240212T114054','20240215T114212', '20240220T111303' , '20240222T105310', '20240228T112459', '20240301T110416', '20240307T121446',
#              '20240212T111123','20240215T113732','20240220T110926','20240222T105243', '20240229T110323', '20240301T105900', '20240307T121323',
#              '20240213T081458','20240214T081320', '20240222T082957', '20240223T090118', '20240229T120111', '20240301T122341']

mouse_list = ['694569','694569','694569', '694569','694569',
              '690164', '690164', '690164','690164', '690164', 
              '699894', '699894','699894', '699894', '699894']
file_list = ['20240301T110416', '20240304T115743', '20240305T092623', '20240306T110105', '20240307T121446',
             '20240301T105900', '20240304T112504', '20240305T092707', '20240306T112647', '20240307T121323',
             '20240301T122341', '20240304T100025', '20240305T080111', '20240306T083350', '20240307T081725']

session_n=0
for file_name, mouse in zip(file_list, mouse_list):
    if session_n == len(mouse_list)/3:
        session_n = 0
    session_n += 1
    path = os.path.join(base_path, mouse, file_name)
    session = file_name[:8]
    session_path = Path(path)
    print(session, mouse)
    try:
        data = parse.load_session_data(session_path)
    except:
        print('Error with loading data')
        
    # try:
    #     if batch == 1:
    #         reward_sites, active_site, encoder_data, config =  parse.parse_data_old(data, path)
    #         print(batch)
    #     else:  
    
    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()

    data['harp_behavior'].streams.OutputSet.load_from_file()
    data['harp_behavior'].streams.OutputClear.load_from_file()
    data['config'].streams['TaskLogic'].load_from_file()
    
    all_epochs = parse.parse_dataframe(data)
    reward_sites = AddExtraColumns(all_epochs, run_on_init=True).reward_sites
    encoder_data = parse.ContinuousData(data).encoder_data

    last_engaged_patch = reward_sites['active_patch'][reward_sites['skipped_count'] >= 10].min()
    if pd.isna(last_engaged_patch):
        last_engaged_patch = reward_sites['active_patch'].max()
    reward_sites = reward_sites.loc[reward_sites['active_patch'] <= last_engaged_patch]

    # except:
    #     print('Error with parsing data')
    #     continue
    
    reward_sites['maximum_reward'] = np.where(reward_sites['odor_label'] == 'Eugenol', 0, 21)
    reward_sites['harvested'] = (reward_sites['maximum_reward']-reward_sites['reward_available'])/21
    reward_sites['harvested_average'] = processing.compute_window(reward_sites, 5, 'harvested', 'active_patch')
    
    maximum = reward_sites.active_patch.max()
    reward_sites['reversed_patch'] = maximum - reward_sites['active_patch'].values
    reward_sites['odor_sites'] = np.arange(len(reward_sites))
    reward_sites['mouse'] = mouse
    reward_sites['session'] = session_n
    summary_df = pd.concat([reward_sites, summary_df], axis=0)

summary_df['odor_label'] = summary_df['patch_label']
summary_df['harvested'] = np.where(summary_df.reward_amount == 0, 0,summary_df['harvested'] )

In [ ]:
color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color1, 'Amyl Acetate': color2, 
                    '2-Heptanone' : color2, 'Methyl Acetate': color1, 'Fenchone': color3, '2,3-Butanedione': color4,
                    'Methyl Butyrate': color1, 'Eugenol': color3}

In [ ]:
mouse =  '694569'
session = 3
choice = 'visit_number'
reward_sites = summary_df.loc[(summary_df['mouse'] == mouse)&(summary_df['session'] ==session)]
reward_sites = reward_sites.loc[reward_sites['odor_label'] != 'Eugenol']
reward_sites['harvested_average'] = processing.compute_window(reward_sites, 5, 'harvested', 'odor_sites')

fig, ax = plt.subplots(figsize=(22, 4))
sns.scatterplot(data=reward_sites, x='odor_sites', y=choice, hue='odor_label', palette=color_dict_label, s=50, ax=ax)
plt.plot(reward_sites['odor_sites'], reward_sites[choice], color='black', linewidth=2, alpha=0.5, zorder=0)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title(f'{mouse} - session {session}')
sns.despine()

In [ ]:
for mouse in ['694569', '690164', '699894']:
    fig, axes = plt.subplots(1,len(summary_df.session.unique()), figsize=(4*len(summary_df.session.unique()), 6), sharey= True)
    for session, ax in zip(sorted(summary_df.session.unique()), axes.flatten()):
        new_df = summary_df.loc[(summary_df.session == session)&(summary_df.mouse == mouse)].groupby(['active_patch', 'reversed_patch', 'odor_label']).agg({'harvested': 'max', 
                                                                                                                                                            'visit_number':'count'}).reset_index()
        sns.histplot(new_df.loc[new_df.odor_label == 'Amyl Acetate']['visit_number'], bins=np.arange(0,8,0.5), color=color_dict_label['Amyl Acetate'], ax=ax)
        sns.histplot(new_df.loc[new_df.odor_label == 'Alpha-pinene']['visit_number'], bins=np.arange(0,8,0.5), color=color_dict_label['Alpha-pinene'], ax=ax)
        sns.despine()
        ax.set_title(f'Session {session}')
        ax.set_xlabel('Visit number')
        ax.set_ylabel('Count')
    plt.suptitle(f'{mouse}')
    plt.tight_layout()

In [ ]:
palette = {'Alpha-pinene': '#1b9e77', 'Amyl Acetate': '#d95f02', 'Eugenol': '#7570b3'}
amount_palette = {3: '#1b9e77', 7: '#d95f02', 0: '#7570b3'}
for mouse in ['694569', '690164', '699894']:
    if mouse == '694569':
        palette = {'Alpha-pinene': '#1b9e77', 'Amyl Acetate': '#d95f02', 'Eugenol': '#7570b3'}
    else:
        palette = {'Alpha-pinene': '#d95f02', 'Amyl Acetate': '#1b9e77', 'Eugenol': '#7570b3'}
        
    fig, ax = plt.subplots(1,len(summary_df.session.unique()), figsize=(4*len(summary_df.session.unique()), 5), sharey= True)
    for session in sorted(summary_df.session.unique()):
        new_df = summary_df.loc[(summary_df.session == session)&(summary_df.mouse == mouse)].groupby(['active_patch', 'reversed_patch', 'odor_label', 'reward_amount']).agg({'harvested': 'max', 'visit_number':'count'}).reset_index()
        for odor_label in ['Amyl Acetate', 'Alpha-pinene', 'Eugenol']:
            df= new_df.loc[new_df.odor_label == odor_label]
            df = df.loc[df.active_patch > 3]
            df = df.loc[df.active_patch < df.active_patch.max() - 10]
            # if session == 1:
            #     patch = 'reversed_patch'
            #     ax[session-1].set_xlim(df.active_patch.max(), 0)
            # elif session == 4:
            #     patch = 'reversed_patch'
            #     ax[session-1].set_xlim(df.active_patch.max(), 0)
            # else:
            #     patch = 'active_patch'
            #     if session == 2:
            #         session_top = df.active_patch.max()
            #     elif session == 3:
            #         df['active_patch'] = session_top + df['active_patch']
                    
            df['harvested_average'] = processing.compute_window(df, 5, 'harvested', patch)
            sns.lineplot(data=df, x=patch, y='visit_number', legend=False, color=palette[odor_label], marker='.', linewidth=1, ax=ax[session-1])
            ax[session-1].set_xlabel('Patch number')
            ax[session-1].set_ylabel('Total stops')
            ax[session-1].set_title(f'Session {session}')
    
    ax[1].yaxis.set_visible(False)
    ax[1].spines['left'].set_color('none')
    ax[2].yaxis.set_visible(False)
    ax[2].spines['left'].set_color('none')
    ax[3].yaxis.set_visible(False)
    ax[3].spines['left'].set_color('none')
    ax[4].yaxis.set_visible(False)
    ax[4].spines['left'].set_color('none')
    # ax[5].yaxis.set_visible(False)
    # ax[5].spines['left'].set_color('none')
    plt.suptitle(f'{mouse}')
    ax[0].locator_params(axis='y', nbins=4)
    sns.despine()
    plt.tight_layout()

In [ ]:
summary_df.loc[(summary_df['mouse'] == mouse)&(summary_df['session'] ==session)]

In [ ]:
mouse =  '694569'
session = 7
reward_sites = summary_df.loc[(summary_df['mouse'] == mouse)&(summary_df['session'] ==session)]
if isinstance(data['config'], dict):
    config = data['config']
else:
    config = data['config'].streams.TaskLogic.data
plotting.segmented_raster_vertical(reward_sites, 
                                config, 
                                color_dict_label=color_dict_label)

## Generalization experiments for batch 1

In [ ]:
import json
def parse_data_here(data):
    try:
        ## Load data from encoder efficiently
        data['harp_behavior'].streams.AnalogData.load_from_file()
        encoder_data = data['harp_behavior'].streams.AnalogData.data
    except:
        encoder_data = pd.DataFrame()
        encoder_data['Encoder']  = parse.read_harp_bin(path + "\Behavior\Register__44"+".bin")[1]

    try:
        # Open and read the JSON file
        with open(str(path)+'\Config\TaskLogic.json', 'r') as json_file:
            config = json.load(json_file)
            
    except:
        with open(str(path)+'\config.json', 'r') as json_file:
            config = json.load(json_file)
        
    try:
        wheel_size = config.streams.Rig.data['treadmill']['wheelDiameter']
        PPR = -config.streams.Rig.data['treadmill']['pulsesPerRevolution']
        
    except:
        wheel_size = 15
        PPR = -8192.0

    perimeter = wheel_size*np.pi
    resolution = perimeter / PPR
    encoder_data['velocity'] = (encoder_data['Encoder'] * resolution)*1000

    # Reindex the seconds so they are aligned to beginning of the session
    start_time = encoder_data.index[0]
    # encoder_data.index -= start_time

    # Get the first odor onset per reward site
    data['software_events'].streams.ActiveSite.load_from_file()
    active_site = data['software_events'].streams.ActiveSite.data

    # Use json_normalize to create a new DataFrame from the 'data' column
    df_normalized = pd.json_normalize(active_site['data'])
    df_normalized.index = active_site.index

    # Concatenate the normalized DataFrame with the original DataFrame
    active_site = pd.concat([active_site, df_normalized], axis=1)

    active_site['label'] = np.where(active_site['label'] == 'Reward', 'RewardSite', active_site['label'])
    active_site.rename(columns={'startPosition':'start_position'}, inplace= True)
    # Rename columns

    active_site = active_site[['label', 'start_position','length']]
    reward_sites = active_site[active_site['label'] == 'RewardSite']

    data['software_events'].streams.GiveReward.load_from_file()
    reward = data['software_events'].streams.GiveReward.data
    reward.fillna(0, inplace=True)

    try:
        data['software_events'].streams.ActivePatch.load_from_file()
        patches = data['software_events'].streams.ActivePatch.data

    except:
        patches = active_site.loc[active_site['label'] == 'InterPatch']
        patches.rename(columns={'label':'name'}, inplace=True)
        patches['name'] = np.where(patches['name'] == 'InterPatch', 'ActivePatch', patches['name'])

    try:
        # Old way of obtaining the reward amount
        reward_available = event[1]["data"]["patchRewardFunction"]["initialRewardAmount"]
    except:
        reward_available = config['environmentStatistics']['patches'][0]['patchRewardFunction']['initialRewardAmount']
                
    reward_updates = pd.concat([patches, reward])
    reward_updates.sort_index(inplace=True)
    reward_updates["current_reward"] = np.nan

    for event in reward_updates.iterrows():
        if event[1]["name"] == 'GiveReward': #update reward
            reward_available -= event[1]["data"]
        elif event[1]["name"] == 'ActivePatch': #reset reward
            try:
                # Old way of obtaining the reward amount
                reward_available = event[1]["data"]["patchRewardFunction"]["initialRewardAmount"]
            except:
                reward_available = config['environmentStatistics']['patches'][0]['patchRewardFunction']['initialRewardAmount']
        else:
            raise ValueError("Unknown event type")
        reward_updates.at[event[0], "current_reward"] = reward_available

    for site in reward_sites.itertuples():
        try:
            arg_min, val_min = processing.find_closest(site.Index, reward_updates.index.values, mode="below_zero")
            reward_sites.loc[site.Index, "reward_available"] = reward_updates["current_reward"].iloc[arg_min]
        except:
            reward_sites.loc[site.Index, "reward_available"] = reward_available

    # Find responses to Reward site
    data['software_events'].streams.ChoiceFeedback.load_from_file()
    choiceFeedback = data['software_events'].streams.ChoiceFeedback.data

    reward_sites.loc[:, "active_patch"] = -1
    reward_sites.loc[:, "visit_number"] = -1
    reward_sites.loc[:, "has_choice"] = False
    reward_sites.loc[:, "reward_delivered"] = 0
    reward_sites.loc[:, "past_no_reward_count"] = 0
    past_no_reward_counter = 0
    current_patch_idx = -1

    visit_number = 0
    for idx, event in enumerate(reward_sites.iterrows()):
        arg_min, val_min = processing.find_closest(event[0], patches.index.values, mode="below_zero")
        if not(np.isnan(arg_min)):
            reward_sites.loc[event[0], "active_patch"] = arg_min
        if current_patch_idx != arg_min:
            current_patch_idx = arg_min
            visit_number = 0
        else:
            visit_number += 1
        reward_sites.loc[event[0], "visit_number"] = visit_number

        if idx < len(reward_sites) - 1:
            choice = choiceFeedback.loc[(choiceFeedback.index >= reward_sites.index[idx]) & (choiceFeedback.index < reward_sites.index[idx+1])]
            reward_in_site = reward.loc[(reward.index >= reward_sites.index[idx]) & (reward.index < reward_sites.index[idx+1])]
        else:
            choice = choiceFeedback.loc[(choiceFeedback.index >= reward_sites.index[idx])]
            reward_in_site = reward.loc[(reward.index >= reward_sites.index[idx])]
        reward_sites.loc[event[0], "has_choice"] = len(choice) > 0
        reward_sites.loc[event[0], "reward_delivered"] = reward_in_site.iloc[0]["data"] if len(reward_in_site) > 0 else 0
        reward_sites.loc[event[0], "past_no_reward_count"] = past_no_reward_counter
        if reward_sites.loc[event[0], "reward_delivered"] == 0 and reward_sites.loc[event[0], "has_choice"] == 1:
            past_no_reward_counter += 1
        else:
            past_no_reward_counter = 0
    try:
        df_patch = pd.json_normalize(patches['data'])
        df_patch.reset_index(inplace=True)
        df_patch.rename(columns={'index':'active_patch', 'label': 'odor_label', 'rewardSpecifications.amount': 'amount'}, inplace=True)
        df_patch.rename(columns={'reward_specification.reward_function.amount.value': 'amount'}, inplace=True)
    except:
        df_patch = pd.DataFrame(columns=['active_patch', 'odor_label', 'amount'])
        df_patch['active_patch'] = np.arange(len(patches))
        df_patch['odor_label'] = config['environmentStatistics']['patches'][0]['label']
        df_patch['amount'] = config['environmentStatistics']['patches'][0]['rewardSpecifications']['amount']
        
    reward_sites = pd.merge(reward_sites.reset_index(),df_patch[['odor_label', 'active_patch', 'amount']],  on='active_patch')

    # Create new column for adjusted seconds to start of session
    reward_sites['adj_seconds'] = reward_sites['Seconds'] - start_time
    reward_sites.index = reward_sites['Seconds']
    reward_sites.drop(columns=['Seconds'], inplace=True)

    # ---------------- Add water triggers times ---------------- #
    data['harp_behavior'].streams.OutputSet.load_from_file()
    water = data['harp_behavior'].streams.OutputSet.data[['SupplyPort0']]
    reward_sites['next_index'] = reward_sites.index.to_series().shift(-1)
    reward_sites['water_onset'] = None

    # Iterate through the actual index of df1
    for value in water.index:
        # Check if the value is between 'Start' and 'End' in df2
        matching_row = reward_sites[(reward_sites.index <= value) & (reward_sites['next_index'].values >= value)]

        # If a matching row is found, update the corresponding row in water with the index value
        if not matching_row.empty:
            matching_index = matching_row.index[0]  # Assuming there's at most one matching row
            reward_sites.at[matching_index, 'water_onset'] = value
            
    # ---------------------------------------------------- #

    # ---------------- Add odor triggers times ---------------- #

    odor_0 = data['harp_behavior'].streams.OutputSet.data['SupplyPort1']
    odor_1 = data['harp_behavior'].streams.OutputSet.data['SupplyPort2']

    odor_0 = odor_0.reset_index()
    odor_1 = odor_1.reset_index()

    odor_0['odor_onset'] = np.where(odor_0['SupplyPort1'] == 1, config['environmentStatistics']['patches'][0]['label'], None)
    odor_1['odor_onset'] = np.where(odor_1['SupplyPort2'] == 1, config['environmentStatistics']['patches'][1]['label'], None)

    odor_df = pd.concat([odor_0[['Time','odor_onset']], odor_1[['Time','odor_onset']]])
    odor_df.sort_index(inplace=True)
    odor_df.dropna(inplace=True)

    odor_df['time_diff'] = odor_df['Time'].diff()
    odor_df = odor_df.drop(index=odor_df.loc[(odor_df['time_diff'] < 1)&(odor_df.index > 0)].index)

    try:
        reward_sites['odor_onset'] = odor_df['Time'].values
    except:
        pass

    # ---------------- Add stop triggers times ---------------- #
    reward_sites['stop_time'] = None

    # Iterate through the actual index of df1
    for value in choiceFeedback.index:
        # Check if the value is between 'Start' and 'End' in df2
        matching_row = reward_sites[(reward_sites.index <= value) & (reward_sites['next_index'].values >= value)]

        # If a matching row is found, update the corresponding row in water with the index value
        if not matching_row.empty:
            matching_index = matching_row.index[0]  # Assuming there's at most one matching row
            reward_sites.at[matching_index, 'stop_time'] = value
            
    reward_sites.drop(columns=['next_index'], inplace=True)
    # ---------------------------------------------------- #

    # Add colum for site number
    reward_sites.loc[:,'total_sites'] = np.arange(len(reward_sites))
    reward_sites.loc[:,'collected'] = np.where((reward_sites['reward_delivered'] != 0), 1, 0)

    reward_sites['next_visit_number'] = reward_sites['visit_number'].shift(-2)
    reward_sites['last_visit'] = np.where(reward_sites['next_visit_number']==0, 1, 0)
    reward_sites.drop(columns=['next_visit_number'], inplace=True)

    reward_sites['last_site'] = reward_sites['visit_number'].shift(-1)
    reward_sites['last_site'] = np.where(reward_sites['last_site'] == 0, 1,0)

    reward_sites['next_patch'] = reward_sites['active_patch'].shift(1)
    reward_sites['next_odor'] = reward_sites['odor_label'].shift(1)
    reward_sites['same_patch'] = np.where((reward_sites['next_patch'] != reward_sites['active_patch'])&(reward_sites['odor_label'] == reward_sites['next_odor'] ), 1, 0)
    reward_sites.drop(columns=['next_patch', 'next_odor'], inplace=True)

    encoder_data = processing.fir_filter(encoder_data, 'velocity', 5)

    return reward_sites, active_site, encoder_data, config

In [ ]:
start_dir = 'Z:/scratch/vr-foraging/data/'
batch = 1
summary_df = pd.DataFrame()
colors_list = sns.palettes.color_palette('husl', 5)

# mouse_list = ['672102','672103','672107']
# file_list = ['20231003T','20231003T','20231003T']
mouse_list = ['672102','672102','672102']
file_list = ['20231002T094747','20231005T103221','20231006T101656']
file_list = ['20231005']

mouse_list = ['672103','672103','672103']
file_list = ['20231004','20231006','20231009']
file_list = ['20231005']

session_n = 0

for file_name, color, mouse in zip(file_list, colors_list, mouse_list):
    print(file_name, mouse)
    
    if session_n == 3:
        session_n = 0
    session_n += 1

    path = processing.find_file(start_dir + mouse+"/", file_name)
    session_path = Path(path)
    try:
        data = parse.load_session_data(session_path)
    except:
        print('Error with loading data')
        
    reward_sites, active_site, encoder_data, config = parse_data_here(data)
    
    # patch_limit = parse.choose_cut(reward_sites, 10)
    # reward_sites = reward_sites.loc[reward_sites.active_patch <= patch_limit]

    if 'Ethyl Butyrate' in reward_sites.odor_label.unique():
        reward_sites['maximum_reward'] = np.where(reward_sites['odor_label'] == 'Ethyl Butyrate', 0, 3)
        reward_sites['harvested'] = (reward_sites['maximum_reward']-reward_sites['reward_available'])/3

    else:
        print(reward_sites.groupby('odor_label')['reward_available'].unique())
        reward_sites['maximum_reward'] = np.where(reward_sites['odor_label'] == 'PineBerries', 0, 6)
        reward_sites['reward_available'] = np.where(reward_sites['odor_label'] == 'PineBerries', 0, reward_sites['reward_available'])
        
        reward_sites['harvested'] = (reward_sites['maximum_reward']-reward_sites['reward_available'])/6

    maximum = reward_sites.active_patch.max()
    reward_sites['reversed_patch'] = maximum - reward_sites['active_patch'].values

    reward_sites['mouse'] = mouse
    reward_sites['session'] = session_n
    summary_df = pd.concat([reward_sites, summary_df], axis=0)

In [ ]:
color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3, 'Bananas': color1, 'PineBerries': color2, 'Pineapple': color3,
                    '2-Heptanone' : color2, 'Methyl Acetate': color1, 'Fenchone': color3, '2,3-Butanedione': color4,
                    'Methyl Butyrate': color1, 'Eugenol':color3, 'Octanol':color3}
if isinstance(data['config'], dict):
    config = data['config']
else:
    config = data['config'].streams.TaskLogic.data
plotting.segmented_raster_vertical(reward_sites, 
                                config, 
                                color_dict_label=color_dict_label)

In [ ]:
reward_sites['odor_sites'] = np.arange(len(reward_sites))

In [ ]:
palette = {'Octanol': 'blue', 'Bananas': 'darkorange', 'Ethyl Butyrate': 'crimson', 'PineBerries': 'indigo'}
fig, ax = plt.subplots(1,len(summary_df.session.unique()), figsize=(4*len(summary_df.session.unique()), 4), sharey= True)
for session in summary_df.session.unique():
    new_df = summary_df.loc[(summary_df.session == session)&(summary_df.mouse == mouse)].groupby(['active_patch', 'reversed_patch', 'odor_label']).agg({'has_choice': 'sum'}).reset_index()
    for odor_label in summary_df.odor_label.unique():
        df= new_df.loc[new_df.odor_label == odor_label]
        if session == 1:
            patch = 'reversed_patch'
            ax[session-1].set_xlim(75, 0)
        elif session == 2:
            patch_number = df.active_patch.max()
            patch = 'active_patch'
        df['harvested_average'] = processing.compute_window(df, 5, 'has_choice', patch)
        sns.lineplot(data=df, x=patch, y='has_choice', legend=False, color=palette[odor_label], marker='.', linewidth=1, ax=ax[session-1])
        ax[session-1].set_xlabel('Patch number')
        ax[session-1].set_ylabel('Total stops')
        ax[session-1].set_title(f'Session {session}')
        

ax[2].yaxis.set_visible(False)
ax[2].spines['left'].set_color('none')
ax[0].locator_params(axis='y', nbins=3)
ax[0].locator_params(axis='x', nbins=4)
ax[1].locator_params(axis='x', nbins=4)
ax[2].locator_params(axis='x', nbins=4)
sns.despine()